In [99]:
 import pandas as pd
from pathlib import Path
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
import hvplot.pandas

In [100]:
csvpath = Path('../Project-03/teamGameStats.csv')
teamstats_df = pd.read_csv(csvpath)

In [101]:
teamstats_df.info('include=all')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1700 entries, 0 to 1699
Data columns (total 24 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   TEAM      1700 non-null   object 
 1   MATCHUP   1700 non-null   object 
 2   GAMEDATE  1700 non-null   object 
 3   W/L       1700 non-null   object 
 4   MIN       1700 non-null   int64  
 5   PTS       1700 non-null   int64  
 6   FGM       1700 non-null   int64  
 7   FGA       1700 non-null   int64  
 8   FG%       1700 non-null   float64
 9   3PM       1700 non-null   int64  
 10  3PA       1700 non-null   int64  
 11  3P%       1700 non-null   float64
 12  FTM       1700 non-null   int64  
 13  FTA       1700 non-null   int64  
 14  FT%       1700 non-null   float64
 15  OREB      1700 non-null   int64  
 16  DREB      1700 non-null   int64  
 17  REB       1700 non-null   int64  
 18  AST       1700 non-null   int64  
 19  TOV       1700 non-null   int64  
 20  STL       1700 non-null   int6

In [102]:
teamstats_df = teamstats_df.drop(columns=['TEAM','MATCHUP','GAMEDATE', 'W/L'])

In [103]:
teamstats_scaled = StandardScaler().fit_transform(teamstats_df)
print(teamstats_scaled[0:5])

[[-0.22424044  1.39287633  0.76382311  0.56422903  0.42595961  0.06949194
  -0.37449775  0.47067759  1.55063575  1.72481131 -0.13135184  1.21146975
   0.48435507  1.06245855 -0.9517666  -0.99793281  0.13949991  0.88429842
  -0.09780737  0.79489375]
 [-0.22424044 -0.22946601 -0.40652325 -0.79847092  0.07156016  1.2949722
   1.1745197   0.62190645 -0.67534997 -0.67676158 -0.13135184 -0.8150259
   0.86025263  0.27682152  0.26706822  0.01008013 -0.54392906  1.30421667
   1.35647125  0.52992917]
 [ 3.66564477  0.01388534 -0.0164078   1.92692898 -1.12220641 -1.64618043
  -1.07859658 -1.66979243  1.20817641  1.15973533  0.20889345  2.94846602
  -0.45538884  1.21958596  0.06392908 -0.24192311 -0.54392906 -0.37545631
  -0.82494667 -0.66241146]
 [-0.22424044 -0.22946601 -0.60158098 -0.1928265  -0.54397573  0.8047801
   0.61124063  0.51720954  0.00956872 -0.1116856   0.30610639  1.7904685
   0.48435507  1.37671337  0.67334649  0.76608983  0.4812144  -1.63521105
  -1.55208598  0.13248229]
 [-0.224

In [104]:
# defining 'W/L' column as feature set
#X = teamstats_df.copy()
#X.drop('W/L', axis=1, inplace=True)
#X.head(20)

In [105]:
# converting 'W/L' to 0/1's
#y=teamstats_df['W/L'].apply(lambda x: 1 if x=="W" else 0)


In [117]:
pca = PCA(n_components=5)

teamstats_pca = pca.fit_transform(teamstats_scaled)

In [119]:
df_teamstats_pca = pd.DataFrame(data=teamstats_pca, columns=['pc01', 'pc02', 'pc03', 'pc04', 'pc05']) #, 'pc06', 'pc07', 'pc08', 'pc09', 'pc10', 'pc11', 'pc12', 'pc13', 'pc14', 'pc15', 'pc16', 'pc17', 'pc18', 'pc19', 'pc20'])
df_teamstats_pca.head()

,pc01,pc02,pc03,pc04,pc05
0,-20.603645,-6.533131,4.731864,-13.308334,9.797052
1,-6.454234,8.111918,2.154247,5.775870,-6.696022
2,11.617374,-9.762855,8.196661,-5.067524,20.579784
3,-2.079443,1.914823,5.477724,5.457933,0.501676
4,18.223632,4.186509,7.812763,-6.172205,7.507980


In [120]:
pca.explained_variance_ratio_

array([0.50389108, 0.14697748, 0.13298775, 0.11914847, 0.09699523])

In [126]:
inertia = []
k = list(range(1,5))

# Calculate the inertia for the range of k values
for i in k:
    km = KMeans(n_clusters=i, random_state=0)
    km.fit(df_teamstats_pca)
    inertia.append(km.inertia_)
    

# Creating the Elbow Curve
elbow_data = {"k": k, "inertia": inertia}
df_elbow = pd.DataFrame(elbow_data)
df_elbow.hvplot.line(x="k", y="inertia", xticks=k, title="Elbow Curve")

:Curve   [k]   (inertia)

In [127]:
 # Predicting clusters with k2

# Initialize the K-Means model
model = KMeans(n_clusters=2, random_state=0)

# Fit the model
model.fit(df_teamstats_pca)

# Predict clusters
predictions = model.predict(df_teamstats_pca)

# Add the predicted class columns
df_teamstats_pca["class"] = model.labels_
df_teamstats_pca.head()

,pc01,pc02,pc03,pc04,pc05,class
0,-20.603645,-6.533131,4.731864,-13.308334,9.797052,1
1,-6.454234,8.111918,2.154247,5.775870,-6.696022,1
2,11.617374,-9.762855,8.196661,-5.067524,20.579784,0
3,-2.079443,1.914823,5.477724,5.457933,0.501676,1
4,18.223632,4.186509,7.812763,-6.172205,7.507980,0


In [128]:
# Plotting the clusters
df_teamstats_pca.hvplot.scatter(
    x="pc01",
    y="pc02",
    hover_cols=["class"],
    by="class",
)

:NdOverlay   [class]
   :Scatter   [pc01]   (pc02,class)